In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#Chargement du dataframe
df_customer = pd.read_csv('./csv/shopping_behavior_updated.csv', sep=";")

u_products = df_customer['Item Purchased'].unique()

data = {}
dataReview = {}

for i, row in df_customer.iterrows():
    items = row['Item Purchased'].split(', ')
    review = row['Review Rating']
    indice = 0
    for item in items:
        if item not in data:
            data[item] = [0] * len(df_customer)
            dataReview[item + "_review"] = [0] * len(df_customer)
        data[item][i] += 1
        if dataReview[item + "_review"][i] == 0:
            dataReview[item + "_review"][i] = review
        else:
            dataReview[item + "_review"][i] = (review + dataReview[item + "_review"][i])/2
    
df_review_table = pd.DataFrame(dataReview)


#On regroupe notre dataframe en fonction des colonnes Age, Gender, Location, Subscription Status, Frequency of Purchases
df_customer.drop(columns=['Item Purchased'], inplace=True)
df_customer.drop(columns=['Customer ID'], inplace=True)
df_customer.drop(columns=['Category'], inplace=True)
df_customer.drop(columns=['Purchase Amount (USD)'], inplace=True)
df_customer.drop(columns=['Previous Purchases'], inplace=True)
df_customer.drop(columns=['Color'], inplace=True)
df_customer.drop(columns=['Review Rating'], inplace=True)
df_customer.drop(columns=['Size'], inplace=True)

df_review_table.to_csv('./csv/review_table.csv', sep=";", index=False)

print(df_customer.head(5))

   Age Gender       Location Subscription Status Frequency of Purchases  \
0   55   Male       Kentucky                 Yes            Fortnightly   
1   19   Male          Maine                 Yes            Fortnightly   
2   50   Male  Massachusetts                 Yes                 Weekly   
3   21   Male   Rhode Island                 Yes                 Weekly   
4   45   Male         Oregon                 Yes               Annually   

   Season  Shipping Type Discount Applied Promo Code Used Payment Method  
0  Winter        Express              Yes             Yes          Venmo  
1  Winter        Express              Yes             Yes           Cash  
2  Spring  Free Shipping              Yes             Yes    Credit Card  
3  Spring   Next Day Air              Yes             Yes         PayPal  
4  Spring  Free Shipping              Yes             Yes         PayPal  


## Encodage des données

In [33]:
import pickle as pk

cols_cat_info = [1,2,3,4,5,6,7,8,9]

#On créer une copie de notre dataframe source
df_grouped_customer = df_customer.copy()

#On supprime la colonne des achats

a_grouped_customer = df_grouped_customer.values
#Encodage des colonnes catégorielles
label_encoders_info = [LabelEncoder() for _ in range(len(cols_cat_info))]
for i, col_idx in enumerate(cols_cat_info):
    a_grouped_customer[:, col_idx] = label_encoders_info[i].fit_transform(a_grouped_customer[:, col_idx])

with open('./pickles/label_encoders_recommandation.pkl', 'wb') as f:
    pk.dump(label_encoders_info, f)
#Print des données encodées
print(a_grouped_customer[0])

a_product_review = df_review_table.values

[55 1 16 1 3 3 1 1 1 5]


In [34]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=4)
neigh.fit(a_grouped_customer, a_product_review)

y_pred = neigh.predict([a_grouped_customer[0]])
print(neigh.predict([a_grouped_customer[0]]))

with open('./pickles/knn_recommandation.pkl', 'wb') as f:
    pk.dump(neigh, f)

[[0.775 0.    0.    1.25  0.    0.    0.    0.    0.    0.    0.    0.
  0.95  0.    0.    0.    0.    0.    0.    0.    0.    0.825 0.    0.
  0.   ]]


In [43]:
df_customer = pd.read_csv('./csv/shopping_behavior_updated.csv', sep=";")

#On récupère l'ensemble des valeurs unique des produits
u_products = df_customer['Item Purchased'].str.split(', ', expand = True).stack().unique()

def getItemsToRecommend(uniqueProducts, idClient, prediction): 
    recommendedItems = []
    alreadyBought = []
    #On récupère les produits déjà achetés
    for i in range(len(a_product_review[idClient])):
        if a_product_review[idClient][i] > 0:
            print(uniqueProducts[i])
            alreadyBought.append(uniqueProducts[i])
            
    #On récupère les produits recommandés
    for j in range(len(prediction[0])):
        if prediction[0][j] > 0:
            #Si le produit n'est pas déjà acheté on l'ajoute à la liste des recommandations
            if uniqueProducts[j] not in alreadyBought:
                recommendedItems.append(uniqueProducts[j])
    return recommendedItems


print(getItemsToRecommend(u_products, 0, y_pred))

Blouse
['Sandals', 'Sunglasses', 'Backpack']
